In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize
from pandas.api.types import is_string_dtype
import os
import json

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression



In [3]:
PATH = '../data/'

## todo:

1. <s> Figure out how to get stuff out of the JSON columns </s>
3. <s> Prepare the dependent variable so it predicts what it needs to </s>
4. <s> Make sure categorical variables are correctly encoded </s>
5. <s> one-hot encode variables with cardinality <7 </s>
6. <s> Get variables out of the date variable </s>
7. Not sure I understand the distribution of "totals.transactionRevenue". Why is the mode around 16? 
7. <s> Deal with missing values </s>
8. Preprocess the test data, too. Run the transformations on the joint df and then divide
9. Fix the date variables
10. get rid of the unimportant features (importance ==0)
11. get rid of features with cardinality of 1 (ie. all values the same)



# Load the data

In [4]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [5]:
# Get data hidden inside json columns out into separate columns 
# credit: julian3833 https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

def load_df(csv_path=f'{PATH}train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str',
                            'totals.transactionRevenu': 'int64'
                           },
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [6]:
nrows=None
# nrows=2000

In [7]:
%time df = load_df(nrows=nrows)

Loaded train.csv. Shape: (903653, 55)
CPU times: user 3min 11s, sys: 4.88 s, total: 3min 16s
Wall time: 3min 18s


In [8]:
df.describe()

,date,visitId,visitNumber,visitStartTime
count,9.036530e+05,9.036530e+05,903653.000000,9.036530e+05
mean,2.016589e+07,1.485007e+09,2.264897,1.485007e+09
std,4.697698e+03,9.022124e+06,9.283735,9.022124e+06
min,2.016080e+07,1.470035e+09,1.000000,1.470035e+09
25%,2.016103e+07,1.477561e+09,1.000000,1.477561e+09
50%,2.017011e+07,1.483949e+09,1.000000,1.483949e+09
75%,2.017042e+07,1.492759e+09,1.000000,1.492759e+09
max,2.017080e+07,1.501657e+09,395.000000,1.501657e+09


In [9]:
#date span
df.date.min(), df.date.max()

(20160801, 20170801)

# Add date variables

In [10]:
# extract date vars. Adapted from the fastai library

def add_datepart(df, fldname, drop=False, time=False):
    
    fld = df[fldname]
    fld_dtype = fld.dtype

    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, format='%Y%m%d')
        
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())
        
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    
    if drop: 
        df.drop(fldname, axis=1, inplace=True)

In [11]:
add_datepart(df, 'date')

In [30]:
[col for col in df.columns]

['channelGrouping_(Other)',
 'channelGrouping_Affiliates',
 'channelGrouping_Direct',
 'channelGrouping_Display',
 'channelGrouping_Organic Search',
 'channelGrouping_Paid Search',
 'channelGrouping_Referral',
 'channelGrouping_Social',
 'channelGrouping_nan',
 'socialEngagementType_Not Socially Engaged',
 'socialEngagementType_nan',
 'device.browserSize_not available in demo dataset',
 'device.browserSize_nan',
 'device.browserVersion_not available in demo dataset',
 'device.browserVersion_nan',
 'device.deviceCategory_desktop',
 'device.deviceCategory_mobile',
 'device.deviceCategory_tablet',
 'device.deviceCategory_nan',
 'device.flashVersion_not available in demo dataset',
 'device.flashVersion_nan',
 'device.language_not available in demo dataset',
 'device.language_nan',
 'device.mobileDeviceBranding_not available in demo dataset',
 'device.mobileDeviceBranding_nan',
 'device.mobileDeviceInfo_not available in demo dataset',
 'device.mobileDeviceInfo_nan',
 'device.mobileDeviceMar

In [11]:
df.to_hdf(f'{PATH}'+"preprocessed_df.hdf", key="df")
df = pd.read_hdf(f'{PATH}'+"preprocessed_df.hdf")


/home/rafal/.conda/envs/fastai/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['channelGrouping', 'fullVisitorId', 'sessionId', 'socialEngagementType', 'device.browser', 'device.browserSize', 'device.browserVersion', 'device.deviceCategory', 'device.flashVersion', 'device.language', 'device.mobileDeviceBranding', 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel', 'device.mobileInputSelector', 'device.operatingSystem', 'device.operatingSystemVersion', 'device.screenColors', 'device.screenResolution', 'geoNetwork.city', 'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.networkLocation', 'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bou

# Converting string variables into category variables.

In [12]:
cat_vars = []
for n,c in df.items():
    if is_string_dtype(c): 
        cat_vars.append(n)
        df[n] = c.astype('category').cat.as_ordered()

# A quick look at the missing values

In [13]:
df.isnull().sum().sort_index()/len(df)

channelGrouping                                      0.000000
date                                                 0.000000
device.browser                                       0.000000
device.browserSize                                   0.000000
device.browserVersion                                0.000000
device.deviceCategory                                0.000000
device.flashVersion                                  0.000000
device.isMobile                                      0.000000
device.language                                      0.000000
device.mobileDeviceBranding                          0.000000
device.mobileDeviceInfo                              0.000000
device.mobileDeviceMarketingName                     0.000000
device.mobileDeviceModel                             0.000000
device.mobileInputSelector                           0.000000
device.operatingSystem                               0.000000
device.operatingSystemVersion                        0.000000
device.s

In [14]:
df = df.replace([np.inf, -np.inf], np.nan)

In [15]:
df['totals.transactionRevenue'].dtype

CategoricalDtype(categories=['10000', '100000000', '100050000', '1000780000',
                  '1001590000', '100220000', '100260000', '100350000',
                  '100440000', '1004490000',
                  ...
                  '99850000', '998750000', '9990000', '99900000', '99950000',
                  '999500000', '99960000', '99970000', '99980000', '99990000'],
                 ordered=True)

In [16]:
df['totals.transactionRevenue'].value_counts(dropna=False)


NaN          892138
16990000        256
18990000        189
33590000        187
44790000        170
13590000        135
55990000        122
19990000        116
15990000         98
15190000         93
19190000         92
10990000         84
59990000         81
24990000         77
79990000         65
27190000         64
27180000         62
33980000         54
39990000         51
67180000         46
1990000          44
28780000         40
21990000         40
35980000         39
30390000         39
17590000         38
31990000         37
27980000         37
30380000         37
12990000         37
              ...  
437070000         1
43710000          1
43720000          1
43780000          1
438020000         1
43080000          1
43850000          1
438540000         1
4390000           1
439000000         1
43930000          1
43950000          1
43640000          1
436300000         1
43610000          1
436020000         1
435910000         1
43590000          1
435860000         1


In [17]:
# df.info(null_counts=True)
cols_with_nulls = []
for col in df.columns: 
    if (df[col].hasnans) & (col != "totals.transactionRevenue"): 
        cols_with_nulls.append(col)
        
cols_with_nulls

['totals.bounces',
 'totals.newVisits',
 'totals.pageviews',
 'trafficSource.adContent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaignCode',
 'trafficSource.isTrueDirect',
 'trafficSource.keyword',
 'trafficSource.referralPath']

In [18]:
df['totals.bounces'].value_counts(dropna=False)

NaN    453023
1      450630
Name: totals.bounces, dtype: int64

In [19]:
for col in cols_with_nulls:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col+'_na'] = pd.isnull(df[col])
        df[col] = df[col].fillna(df[col].median())
    elif pd.api.types.is_categorical_dtype(df[col]):
        df[col+'_na'] = pd.isnull(df[col])
        df[col] =  df[col].cat.add_categories(["-1"])
        df[col] = df[col].fillna("-1")




In [20]:
cols_with_nulls = []
for col in df.columns: 
    if df[col].hasnans: 
        cols_with_nulls.append(col)
        
cols_with_nulls

['totals.transactionRevenue']

In [21]:
df['totals.transactionRevenue'].value_counts(dropna=False)

NaN          892138
16990000        256
18990000        189
33590000        187
44790000        170
13590000        135
55990000        122
19990000        116
15990000         98
15190000         93
19190000         92
10990000         84
59990000         81
24990000         77
79990000         65
27190000         64
27180000         62
33980000         54
39990000         51
67180000         46
1990000          44
28780000         40
21990000         40
35980000         39
30390000         39
17590000         38
31990000         37
27980000         37
30380000         37
12990000         37
              ...  
437070000         1
43710000          1
43720000          1
43780000          1
438020000         1
43080000          1
43850000          1
438540000         1
4390000           1
439000000         1
43930000          1
43950000          1
43640000          1
436300000         1
43610000          1
436020000         1
435910000         1
43590000          1
435860000         1


# One-hot encoding categorical variables with cardinality <= 15.

In [22]:
def dummify(df, min_cardinality=0, max_cardinality=15):

    to_dummify = []
    
    for n,c in df.items():
        
        if str(df.dtypes[n]) == 'category':
            if ((len(df[n].cat.categories)) > min_cardinality) \
            & ((len(df[n].cat.categories)) <= max_cardinality):
                to_dummify.append(n)
    print(to_dummify)            
    dummified = pd.get_dummies(df[to_dummify], dummy_na=True)
    dummified = pd.concat([dummified, df], axis=1)
    dummified_df = dummified.drop(to_dummify, axis=1)
    
    return to_dummify, dummified_df

In [23]:
dumd_vars, df = dummify(df)

['channelGrouping', 'socialEngagementType', 'device.browserSize', 'device.browserVersion', 'device.deviceCategory', 'device.flashVersion', 'device.language', 'device.mobileDeviceBranding', 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel', 'device.mobileInputSelector', 'device.operatingSystemVersion', 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.bounces', 'totals.newVisits', 'totals.visits', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.criteriaParameters', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.page', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.campaignCode', 'trafficSource.isTrueDirect', 'trafficSource.medium']


In [25]:
#These are the remaining, not dummified categorical variables.
#Let's convert them to ints (otherwise sklearn's random forest is grumpy)

remaining_cat_vars = (list(set(cat_vars) - set(dumd_vars))) 

for var in remaining_cat_vars:
    if (var != "totals.transactionRevenue"):
        df[var] = df[var].cat.codes

# Prepare the dependent variable (log-transform)

In [27]:
# Prepare the dependent variable (log)

df['totals.transactionRevenue'] = pd.to_numeric(df['totals.transactionRevenue'], errors='coerce')
df['totals.transactionRevenue'] = df['totals.transactionRevenue'].fillna(0)
df["totals.transactionRevenue"] = np.log1p(df["totals.transactionRevenue"])

In [28]:
df['totals.transactionRevenue'].value_counts(dropna=False)

0.000000     892138
16.648136       256
16.759423       189
17.329739       187
17.617495       170
16.424845       135
17.840684       122
16.810743       116
16.587474        98
16.536148        93
16.769900        92
16.212496        84
17.909688        81
17.033986        77
18.197412        65
17.118360        64
17.117992        62
17.341283        54
17.504140        51
18.022886        46
14.503646        44
16.906098        40
17.175191        40
17.398474        39
17.229624        39
16.682841        38
16.379690        37
17.280934        37
17.229295        37
17.147001        37
              ...  
18.649409         1
19.728473         1
19.533590         1
18.469090         1
20.506129         1
19.525109         1
17.498372         1
20.864418         1
19.796243         1
18.478950         1
17.296751         1
18.811979         1
19.823070         1
18.539796         1
18.902675         1
18.690173         1
16.332400         1
17.300130         1
20.428680         1


# Save the preprocessed dataset

In [29]:
df.to_hdf(f'{PATH}'+"preprocessed_df.hdf", key="df")

# _tmp